<a href="https://colab.research.google.com/github/YeswanthSubhash/MovieRecommender/blob/main/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Load the data into a Pandas dataframe
data = pd.read_csv('ratings.csv')

# Define the Reader object to parse the data
reader = Reader(rating_scale=(1, 5))

# Load the data into the Dataset object
dataset = Dataset.load_from_df(data[['user_id', 'movie_id', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(dataset, test_size=0.25)

# Train the SVD algorithm on the training set
algorithm = SVD()
algorithm.fit(trainset)

# Test the algorithm on the test set and print the RMSE
predictions = algorithm.test(testset)
print('RMSE:', rmse(predictions))

# Use the trained algorithm to generate movie recommendations for a given user
user_id = 100
movies = data['movie_id'].unique()
user_movies = data[data['user_id'] == user_id]['movie_id']
new_movies = list(set(movies) - set(user_movies))
testset = [[user_id, movie_id, 4.] for movie_id in new_movies]
predictions = algorithm.test(testset)
recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)

# Print the top 10 movie recommendations for the user
print('Top 10 recommendations for user', user_id, ':')
for recommendation in recommendations[:10]:
    print(recommendation.iid, '-', recommendation.est)


RMSE: 0.8727
RMSE: 0.8727370211039466
Top 10 recommendations for user 100 :
720 - 4.8353146341893085
904 - 4.835216642820601
1148 - 4.749400253606174
112852 - 4.7036676825276835
56782 - 4.703146482344025
750 - 4.678859896541974
1172 - 4.627495803126688
48516 - 4.618755820976956
541 - 4.615787612222366
741 - 4.61200740400235


In [ ]:

import pickle
# Save the model to a file
with open('model.pkl', 'wb') as f:
    pickle.dump(algorithm, f)

In [ ]:
# Use the trained algorithm to generate movie recommendations for a given user
user_id = 101

# Find the movies that the user has already rated
user_movies = data[data['user_id'] == user_id]['movie_id']

# Find the movies that the user has not rated yet
movies = data['movie_id'].unique()
new_movies = list(set(movies) - set(user_movies))

# Create a test set for the user with a high rating for all new movies
testset = [[user_id, movie_id, 4.] for movie_id in new_movies]

# Get the predicted ratings for the new movies
predictions = algorithm.test(testset)

# Sort the predicted ratings in descending order and select the top 5 recommendations
recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

movies =[]
# Print the top 5 movie recommendations for the user
print('Top 5 recommendations for user', user_id, ':')
for recommendation in recommendations:
    print(recommendation.iid, '-', recommendation.est)
    movies.append(recommendation.iid)


Top 5 recommendations for user 101 :
904 - 4.675532666528013
4973 - 4.632928264009423
1203 - 4.535740999527034
1197 - 4.519773476446018
593 - 4.472811731784702


In [ ]:
movies

df = pd.read_csv('movies.csv')
df[df['movieId'].isin(movies)]

,movieId,title,genres
510,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
686,904,Rear Window (1954),Mystery|Thriller
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
905,1203,12 Angry Men (1957),Drama
3622,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance
